In [185]:
#imports
import pandas as pd
from scipy import stats

In [114]:
df = pd.read_csv("data/toolwindow_data.csv")
#let's change our timestamp to a datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.sort_values(by=['user_id', 'timestamp'],ascending=[True, True], inplace=True)

In [178]:
#splitting dataframe to two separate dataframes depending on whether window has been opened automatically or manually (using the vector method for optimization)
df_shifted = df.shift(-1)
#creating masks
mask_opened = df['event'] == 'opened'
mask_closed = df_shifted['event'] == 'closed'
mask_same_user_id = df['user_id'] == df_shifted['user_id']
valid_mask = mask_same_user_id & mask_closed & mask_same_user_id
#creating new dataframe with ONLY those rows with "opened" which are the beginning of correct pair
#meaning (after those records with "opened" are records with "closed"
df_pairs = df[valid_mask].copy()
#applying closing timestamps from shifted DataFrame using valid mask, so every row has its own opening and closing timestamp
df_pairs['timestamp_closed'] = df_shifted[valid_mask]['timestamp']
df_pairs['length'] = df_pairs['timestamp_closed'] - df_pairs['timestamp']

#splitting into two dataframes: one for auto, other for manual openings
final_columns = ['user_id', 'length']
df_auto = df_pairs[df_pairs['open_type'] == 'auto'][final_columns].reset_index(drop=True)
df_non_auto = df_pairs[df_pairs['open_type'] == 'manual'][final_columns].reset_index(drop=True)
df_auto['length'], df_non_auto['length'] = df_auto['length'].dt.total_seconds(), df_non_auto['length'].dt.total_seconds()



In [179]:
df_auto = df_auto.groupby('user_id')['length'].mean().reset_index()
df_auto

,user_id,length
0,1,643.954000
1,2,58.624000
2,3,3140.815105
3,4,27777.630778
4,5,8841.726000
...,...,...
161,194,7.957000
162,195,6.836000
163,196,33.035000
164,198,125.748000


In [180]:
df_non_auto = df_non_auto.groupby('user_id')['length'].mean().reset_index()
df_non_auto

,user_id,length
0,1,3526.676625
1,2,53.363500
2,4,13.592000
3,5,25525.585400
4,6,295.383667
...,...,...
139,191,3.217000
140,192,9.439000
141,199,22.863000
142,200,34.045000


In [181]:
df_merge = pd.merge(
    df_auto,
    df_non_auto,
    on='user_id',
    how='outer',
    suffixes=('_auto', '_non_auto')
)
df_merge

,user_id,length_auto,length_non_auto
0,1,643.954000,3526.676625
1,2,58.624000,53.363500
2,3,3140.815105,NaN
3,4,27777.630778,13.592000
4,5,8841.726000,25525.585400
...,...,...,...
192,198,125.748000,NaN
193,199,NaN,22.863000
194,200,NaN,34.045000
195,201,NaN,8.308000


In [182]:
df_merge['percentage_change'] = round((df_merge['length_auto']/df_merge['length_non_auto']*100),2)
df_merge.dropna(inplace=True)
df_merge

,user_id,length_auto,length_non_auto,percentage_change
0,1,643.954000,3526.676625,18.26
1,2,58.624000,53.363500,109.86
3,4,27777.630778,13.592000,204367.50
4,5,8841.726000,25525.585400,34.64
5,6,7413.007778,295.383667,2509.62
...,...,...,...,...
178,182,650.923250,2.265857,28727.46
183,188,12.539000,2.478000,506.01
184,189,32.562900,14.400500,226.12
185,190,83974.661000,227.935000,36841.49


In [183]:
df_merge = df_merge[df_merge['percentage_change']>100]
df_merge

,user_id,length_auto,length_non_auto,percentage_change
1,2,58.624000,53.363500,109.86
3,4,27777.630778,13.592000,204367.50
5,6,7413.007778,295.383667,2509.62
6,7,39479.047000,6.448000,612268.10
7,8,4340.351944,148.362500,2925.50
...,...,...,...,...
175,179,72.710000,10.982000,662.08
178,182,650.923250,2.265857,28727.46
183,188,12.539000,2.478000,506.01
184,189,32.562900,14.400500,226.12


In [184]:
df_merge.loc[:, 'percentage_change'] = df_merge.loc[:, 'percentage_change'] - 100
df_merge

,user_id,length_auto,length_non_auto,percentage_change
1,2,58.624000,53.363500,9.86
3,4,27777.630778,13.592000,204267.50
5,6,7413.007778,295.383667,2409.62
6,7,39479.047000,6.448000,612168.10
7,8,4340.351944,148.362500,2825.50
...,...,...,...,...
175,179,72.710000,10.982000,562.08
178,182,650.923250,2.265857,28627.46
183,188,12.539000,2.478000,406.01
184,189,32.562900,14.400500,126.12


In [190]:
df_testable = df_merge.copy()
try:
    p_value = stats.wilcoxon(
        df_testable['length_auto'],
        df_testable['length_non_auto']
    )

except ValueError as e:
    print("identical samples")
print(p_value)
median_change = df_testable['percentage_change'].median()
print(f"\n{median_change:.2f}%")

WilcoxonResult(statistic=np.float64(0.0), pvalue=np.float64(3.732014234846569e-16))

1052.05%
